In [1]:
import visa                             # Import PyVISA library
import time
#import VISAresourceExtentions
#import numpy as np

In [2]:
# Initialization

rm = visa.ResourceManager()             # Create Resource Manager object
rs = rm.list_resources()                # Method to list the available resources
print(rs[0])                            
counter = rm.open_resource(rs[0])       # Assign the returned object to the instrument variable (i.e. counter)

USB0::5355::145::457158::0::INSTR


In [3]:
# Initial settings

print(counter.query('*IDN?'))           # Query the Identification string 
counter.write('*RST;*CLS')              # Reset the instrument, clear the Error queue
counter.timeout = 1e7                   # Acquisition timeout (ms) - set it higher than the acquisition time
counter.query('SYST:ERR?')              # Error Checking 

PENDULUM, CNT-91, 457158, V1.31 06 Mar 2015 12:55



'0,"No error"\n-'

In [4]:
#Basic settings

counter.write("FUNC 'FREQ 1'")                          # Setup for frequency measurement from channel A (1) 
counter.write('INP:LEV:AUTO OFF; :INP:LEV 0')           # Enable/Disable autotrigger on channel A; Trigger level (V)
counter.query('SYST:ERR?')                              # Error Checking                         

'0,"No error"\n-'

In [5]:
#counter.write('*ESE 1;*SRE 32')                     # To be developed
#counter.wait_on_event(counter, RQS, 10000)
#counter.last_status
#counter.write('*ESR?')
#counter.read()
#counter.query('SYST:ERR?')

In [6]:
# Calculate MEAN/MAX/MIN/SDEV/ADEV values (ascii format), only if 'CALC:AVER:STAT ON'

counter.write('CALC:AVER:STAT ON')                                        # Enable/disable statitics; 
counter.write('CALC:AVER:TYPE MEAN')                                      # Select the statistical function to be performed (MIN/MAX/MEAN/SDEV/ADEV)
counter.write('FORMAT:TINF ON; :FORMAT ASCII')                            # Read timestamp of each measurement (can also be omitted with CONF and MEAS); Redout in ASCII/REAL mode -> [freq (Hz), tstamp (sec)]
counter.write('SENSE:ACQ:APER 2e-8; :CALC:AVER:COUNT 1e3')                # Gate time (sec) - (In statistics mode is the Pacing time of the samples), minimum: 20ns - Dead time = 6-8 microsec; Set the number of samples to use in statistics sampling                                      
#time.sleep(1)                                                             # Wait (sec)                                                           

start = time.time()

for i in range(1):
    
    counter.query('INIT;*OPC?')                                           # Initialize acquisition; *OPC? query waits until the acquisition ends 
   
    #counter.write('CALC:DATA?')                                 
    #counter.read()
    #value = counter.query('CALC:DATA?')                                   # Fetch list with string characters as its elements
    #value = counter.query_ascii_values('CALC:DATA?')                      # Fetch ASCII value ( ValueError: 'could not convert string to float can occur' ), sometimes doesn't work when timestamp is printed (i.e. with MIN/MAX )         
    value = counter.query_ascii_values('CALC:DATA?', converter = 's')     # Fetch and Convert ASCII value into a string

    value[-1] = value[-1].split('\n')[0]                                  # Delete in the string every character after the number ( i.e. \n0E..)
    value[0] = float(value[0])                                            # Convert string into float number
    value[1] = float(value[1])                                            # Condition necessary when the last element of the list happens to be an empty string = '' 

    print(value)                                                          # List format -> [value, timestamp]
    
end = time.time()
print(end-start)                                                          # Total acquisition time (sec)
counter.query('SYST:ERR?')                                                # Error Checking 

[38864000.0, 0.0]
0.6394293308258057


'-213,"Init ignored"\n'

In [7]:
# Perform  M ('ARM:COUNT') block measurements of N ('TRIG:COUNT') samples each (Dead time = 6-8 microsec)
# Fetch the data of one acquisiton ( Total number of samples/measurements = M*N )

counter.write('CALC:AVER:STAT OFF')                     # Enable/disable statitics; 
counter.write('FORMAT:TINF ON; :FORMAT ASCII')          # Read timestamp of each measurement (can also be omitted with CONF and MEAS); Redout in ASCII/REAL mode -> [freq (Hz), tstamp (sec)]
counter.write('SENSE:ACQ:APER 2e-8')                    # Gate time (sec), minimum: 20ns - Dead time = 6-8 microsec; Set the number of samples to use in statistics sampling                                      
counter.write('TRIG:COUNT 1e3; :ARM:COUNT 1e1')         # Measure N samples inside one block (Triggerings); Number of blocks (Armings)
counter.query('SYST:ERR?')                              # Error Checking  
#time.sleep(1)                                           # Wait (sec)

'0,"No error"\nn'

In [8]:
# Fetch the data in one cycle 

counter.query('INIT;*OPC?')                                                       # Initialize acquisition; *OPC? query waits until the acquisition ends

start = time.time()

#counter.write('FETCH:ARR? 1e3')                   
#counter.read()                                           
#freqtimelist = counter.query('FETCH:ARR? 1e3')                                    # Fetch list with string characters as its elements
#freqtimelist = counter.query_ascii_values('FETCH:ARR? 1e3')                       # Fetch list of ASCII values ( ValueError: 'could not convert string to float can occur' )            
freqtimelist = counter.query_ascii_values('FETCH:ARR? 1e3', converter = 's')      # Fetch and Convert list of ASCII values into a list of strings (Maximum Fetchable samples = 1e4)
    
if  freqtimelist[-1] == '' :    del freqtimelist[-1]                              # Delete the last element of the list when it is an empty string = '' 

freqtimelist[-1] = freqtimelist[-1].split('\n')[0]                                # Delete in the last element of the list of strings every character after the number ( i.e. \n0E..)
freqtimelist = [float(i) for i in freqtimelist]                                   # Convert list of strings into float numbers

print(freqtimelist)

end = time.time()
print(end-start)                                                                  # Total acquisition time (sec)
counter.query('SYST:ERR?')                                                        # Error Checking  

[38810000.0, 38.09852668625, 38810000.0, 38.09853281392, 38850000.0, 38.09853894175, 38830000.0, 38.098545018, 38790000.0, 38.09855114576, 38790000.0, 38.09855727353, 38870000.0, 38.09856515241, 38780000.0, 38.09857318583, 38920000.0, 38.09858121944, 38860000.0, 38.09858925301, 38840000.0, 38.09859728666, 38910000.0, 38.09860532028, 38860000.0, 38.09861335376, 38930000.0, 38.09862138736, 38860000.0, 38.0986294207, 38910000.0, 38.09863745419, 38840000.0, 38.09864548756, 38910000.0, 38.09865352096, 38930000.0, 38.09866155429, 38840000.0, 38.09866958761, 38920000.0, 38.098677621, 38830000.0, 38.09868560273, 38770000.0, 38.09869363613, 38860000.0, 38.09870166942, 38860000.0, 38.09870970272, 38840000.0, 38.09871773616, 38870000.0, 38.09872576942, 38860000.0, 38.09873380265, 38830000.0, 38.09874183602, 38840000.0, 38.09874986928, 38840000.0, 38.09875790257, 38820000.0, 38.09876593587, 38880000.0, 38.09877396919, 38860000.0, 38.09878200239, 38840000.0, 38.09879003573, 38910000.0, 38.098798068

'-213,"Init ignored"\n'

In [9]:
# Fetch the data over multiple cycles 

counter.query('INIT;*OPC?')                                                             # Initialize acquisition; *OPC? query waits until the acquisition ends

start = time.time()

for i in range(10):
        
    #counter.write('FETCH:ARR? 1e3')                   
    #counter.read()                                                  
    #freqtimelist = counter.query('FETCH:ARR? 1e3')                                     # Fetch list with string characters as its elements
    #freqtimelist = counter.query_ascii_values('FETCH:ARR? 1e3')                        # Fetch list of ASCII values ( ValueError: 'could not convert string to float can occur' )            
    freqtimelist = counter.query_ascii_values('FETCH:ARR? 1e3', converter = 's')       # Fetch and Convert list of ASCII values into a list of strings (Maximum Fetchable samples = 1e4)
    
    if  freqtimelist[-1] == '' :    del freqtimelist[-1]                               # Delete the last element of the list when it is an empty string = ''

    freqtimelist[-1] = freqtimelist[-1].split('\n')[0]                                 # Delete in the last element of the list of strings every character after the number ( i.e. \n0E..)
    freqtimelist = [float(i) for i in freqtimelist]                                    # Convert list of strings into float numbers                                 
        
    print(freqtimelist)

    if (i==0): freqtimeList = freqtimelist                                             # Create a bigger List appending all the fetched data lists during the loop 
    else:  freqtimeList = freqtimeList + freqtimelist 
        
    end = time.time()
    print(end-start)                                                                   # Single acquisition time (sec)
    
end = time.time()
print(end-start)                                                                       # Total acquisition time (sec)
counter.query('SYST:ERR?')                                                             # Error Checking  

[38870000.0, 39.52034790211, 38900000.0, 39.52035402998, 38880000.0, 39.52036103309, 38840000.0, 39.52036906612, 38870000.0, 39.5203770991, 38910000.0, 39.52038513217, 38800000.0, 39.52039316517, 38880000.0, 39.52040119844, 38900000.0, 39.52040923183, 38840000.0, 39.52041726539, 38870000.0, 39.520425299, 38860000.0, 39.52043333257, 38890000.0, 39.52044136622, 38870000.0, 39.52044939967, 38870000.0, 39.52045743325, 38850000.0, 39.5204654667, 38920000.0, 39.52047350016, 38850000.0, 39.52048153353, 38800000.0, 39.52048956692, 38870000.0, 39.52049754882, 38870000.0, 39.52050558215, 38790000.0, 39.52051361547, 38880000.0, 39.52052164884, 38870000.0, 39.52052968211, 38830000.0, 39.52053771542, 38870000.0, 39.52054574886, 38860000.0, 39.52055378207, 38770000.0, 39.52056181538, 38890000.0, 39.52056984875, 38880000.0, 39.52057788204, 38790000.0, 39.52058591531, 38890000.0, 39.52059394873, 38860000.0, 39.52060198188, 38850000.0, 39.5206100152, 38910000.0, 39.52061804856, 38890000.0, 39.520626081

[38840000.0, 39.52837618025, 38850000.0, 39.52838421352, 38940000.0, 39.52839224691, 38810000.0, 39.52840028012, 38840000.0, 39.52840831353, 38870000.0, 39.52841634684, 38900000.0, 39.52842438014, 38850000.0, 39.52843241337, 38900000.0, 39.5284404467, 38880000.0, 39.52844847998, 38860000.0, 39.52845651323, 38970000.0, 39.52846454665, 38920000.0, 39.52847257982, 38930000.0, 39.52848061307, 38870000.0, 39.52848864631, 38870000.0, 39.52849667951, 38870000.0, 39.5285047127, 38830000.0, 39.52851274599, 38880000.0, 39.52852077923, 38870000.0, 39.52852881236, 38840000.0, 39.52853684557, 38860000.0, 39.52854487881, 38910000.0, 39.52855291207, 38830000.0, 39.52856089368, 38840000.0, 39.52856892687, 38850000.0, 39.52857695998, 38890000.0, 39.52858499327, 38840000.0, 39.52859302637, 38860000.0, 39.52860105956, 38840000.0, 39.52860909267, 38870000.0, 39.5286171259, 38900000.0, 39.528625159, 38830000.0, 39.52863319209, 38810000.0, 39.52864122518, 38840000.0, 39.5286492584, 38890000.0, 39.5286572915

[38880000.0, 39.53640744222, 38810000.0, 39.53641547531, 38900000.0, 39.53642350854, 38870000.0, 39.53643149017, 38850000.0, 39.53643952325, 38840000.0, 39.53644755644, 38820000.0, 39.53645558956, 38910000.0, 39.5364636227, 38790000.0, 39.53647165581, 38860000.0, 39.5364796889, 38860000.0, 39.53648772204, 38870000.0, 39.53649575517, 38850000.0, 39.53650378824, 38950000.0, 39.53651182138, 38850000.0, 39.53651985438, 38820000.0, 39.53652788747, 38920000.0, 39.53653592058, 38880000.0, 39.53654395367, 38810000.0, 39.536551987, 38890000.0, 39.53656002033, 38910000.0, 39.53656805368, 38820000.0, 39.53657608704, 38850000.0, 39.53658412035, 38930000.0, 39.5365921538, 38810000.0, 39.53660018705, 38900000.0, 39.53660822037, 38920000.0, 39.5366162536, 38820000.0, 39.53662428684, 38840000.0, 39.53663232012, 38830000.0, 39.53664030178, 38880000.0, 39.53664833506, 38810000.0, 39.53665636816, 38930000.0, 39.53666440146, 38850000.0, 39.53667243457, 38840000.0, 39.5366804677, 38880000.0, 39.53668850095

[38880000.0, 39.54443865299, 38850000.0, 39.54444668621, 38880000.0, 39.54445471949, 38840000.0, 39.54446275265, 38830000.0, 39.54447078591, 38850000.0, 39.54447881917, 38960000.0, 39.54448685236, 38840000.0, 39.54449488556, 38900000.0, 39.54450286723, 38820000.0, 39.54451090026, 38850000.0, 39.54451893353, 38770000.0, 39.54452696666, 38860000.0, 39.54453499991, 38950000.0, 39.54454303313, 38800000.0, 39.5445510662, 38840000.0, 39.54455909931, 38890000.0, 39.54456713251, 38800000.0, 39.54457516567, 38860000.0, 39.54458319878, 38870000.0, 39.54459123191, 38850000.0, 39.54459926502, 38940000.0, 39.54460729821, 38860000.0, 39.54461533125, 38910000.0, 39.54462336439, 38870000.0, 39.54463139746, 38870000.0, 39.54463943058, 38870000.0, 39.54464746364, 38810000.0, 39.54465549662, 38870000.0, 39.54466352981, 38850000.0, 39.54467156279, 38770000.0, 39.54467959592, 38920000.0, 39.54468762902, 38870000.0, 39.54469566199, 38920000.0, 39.54470369505, 38850000.0, 39.54471172807, 38900000.0, 39.54471

[38880000.0, 39.55246985981, 38860000.0, 39.55247789288, 38800000.0, 39.55248592593, 38910000.0, 39.5524939591, 38840000.0, 39.55250199211, 38800000.0, 39.55251002512, 38890000.0, 39.55251805826, 38860000.0, 39.55252609129, 38840000.0, 39.55253412434, 38870000.0, 39.55254215735, 38890000.0, 39.55255019037, 38860000.0, 39.55255822338, 38820000.0, 39.55256625639, 38850000.0, 39.55257428948, 38900000.0, 39.55258232244, 38800000.0, 39.55259035552, 38910000.0, 39.55259833727, 38830000.0, 39.5526063706, 38830000.0, 39.55261440413, 38800000.0, 39.5526224378, 38830000.0, 39.55263047137, 38830000.0, 39.55263850491, 38890000.0, 39.55264653854, 38880000.0, 39.55265457197, 38810000.0, 39.55266260543, 38870000.0, 39.55267063896, 38910000.0, 39.55267867235, 38770000.0, 39.5526867057, 38880000.0, 39.55269473908, 38900000.0, 39.55270277241, 38800000.0, 39.55271080575, 38880000.0, 39.55271883912, 38930000.0, 39.55272687245, 38800000.0, 39.55273490572, 38930000.0, 39.55274293912, 38830000.0, 39.55275097

...

[38850000.0, 39.56853228207, 38820000.0, 39.56854031525, 38920000.0, 39.56854834859, 38860000.0, 39.56855638171, 38850000.0, 39.56856441486, 38890000.0, 39.56857244814, 38870000.0, 39.56858048131, 38860000.0, 39.56858851448, 38890000.0, 39.56859654765, 38850000.0, 39.56860458083, 38840000.0, 39.56861261401, 38970000.0, 39.5686206473, 38930000.0, 39.56862868033, 38890000.0, 39.56863671347, 38860000.0, 39.5686447466, 38940000.0, 39.56865277983, 38870000.0, 39.56866081282, 38800000.0, 39.56866884597, 38880000.0, 39.56867687911, 38870000.0, 39.5686849121, 38830000.0, 39.56869294527, 38830000.0, 39.56870097832, 38880000.0, 39.56870901147, 38960000.0, 39.56871704465, 38860000.0, 39.56872507765, 38860000.0, 39.56873311081, 38870000.0, 39.56874109256, 38750000.0, 39.56874912585, 38860000.0, 39.56875715933, 38860000.0, 39.56876519264, 38820000.0, 39.568773226, 38940000.0, 39.56878125944, 38850000.0, 39.56878929265, 38900000.0, 39.56879732599, 38850000.0, 39.5688053593, 38840000.0, 39.5688133924

[38920000.0, 39.57656354173, 38840000.0, 39.57657157501, 38910000.0, 39.57657960838, 38870000.0, 39.57658764172, 38850000.0, 39.57659567505, 38790000.0, 39.57660365692, 38840000.0, 39.57661169023, 38880000.0, 39.57661972357, 38850000.0, 39.57662775681, 38830000.0, 39.57663579003, 38840000.0, 39.57664382329, 38820000.0, 39.5766518565, 38820000.0, 39.57665988974, 38880000.0, 39.57666792295, 38840000.0, 39.5766759561, 38880000.0, 39.57668398933, 38860000.0, 39.57669202248, 38820000.0, 39.57670005572, 38870000.0, 39.57670808894, 38860000.0, 39.576716122, 38880000.0, 39.57672415522, 38890000.0, 39.57673218839, 38900000.0, 39.57674022155, 38930000.0, 39.57674825471, 38860000.0, 39.57675628782, 38830000.0, 39.57676432095, 38880000.0, 39.57677235409, 38850000.0, 39.57678038718, 38900000.0, 39.57678842035, 38890000.0, 39.57679645344, 38870000.0, 39.57680448664, 38890000.0, 39.57681246819, 38850000.0, 39.57682050118, 38840000.0, 39.57682853429, 38790000.0, 39.57683656738, 38840000.0, 39.57684460

[38860000.0, 39.58459475394, 38880000.0, 39.58460278715, 38920000.0, 39.58461082023, 38860000.0, 39.58461885329, 38840000.0, 39.58462688637, 38860000.0, 39.58463491953, 38900000.0, 39.5846429526, 38870000.0, 39.58465098576, 38880000.0, 39.58465901884, 38830000.0, 39.58466705181, 38860000.0, 39.58467503343, 38860000.0, 39.58468306651, 38850000.0, 39.58469109956, 38920000.0, 39.58469913264, 38810000.0, 39.58470716574, 38860000.0, 39.58471519875, 38850000.0, 39.5847232318, 38860000.0, 39.58473126479, 38870000.0, 39.58473929784, 38890000.0, 39.58474733092, 38820000.0, 39.58475536389, 38840000.0, 39.58476339691, 38930000.0, 39.58477142993, 38870000.0, 39.58477946295, 38860000.0, 39.58478749623, 38880000.0, 39.58479552968, 38890000.0, 39.58480356326, 38870000.0, 39.5848115968, 38920000.0, 39.58481963041, 38860000.0, 39.58482766394, 38850000.0, 39.58483569746, 38870000.0, 39.58484373105, 38900000.0, 39.58485176447, 38860000.0, 39.58485979792, 38880000.0, 39.5848678313, 38880000.0, 39.58487581

[38830000.0, 39.59262595855, 38910000.0, 39.59263399211, 38820000.0, 39.59264202569, 38980000.0, 39.59265005939, 38880000.0, 39.59265809289, 38820000.0, 39.59266612647, 38830000.0, 39.59267415995, 38860000.0, 39.59268219339, 38840000.0, 39.59269022691, 38860000.0, 39.59269826028, 38840000.0, 39.59270629369, 38870000.0, 39.59271432702, 38910000.0, 39.59272236037, 38840000.0, 39.59273039373, 38800000.0, 39.59273842705, 38920000.0, 39.59274646039, 38840000.0, 39.59275444218, 38810000.0, 39.59276247549, 38890000.0, 39.59277050877, 38830000.0, 39.59277854211, 38800000.0, 39.59278657546, 38850000.0, 39.59279460873, 38850000.0, 39.59280264202, 38840000.0, 39.59281067534, 38820000.0, 39.59281870862, 38860000.0, 39.59282674195, 38790000.0, 39.59283477521, 38890000.0, 39.59284280855, 38890000.0, 39.59285084179, 38880000.0, 39.59285887496, 38890000.0, 39.59286690827, 38830000.0, 39.59287494147, 38830000.0, 39.59288297467, 38900000.0, 39.59289100797, 38920000.0, 39.59289904123, 38840000.0, 39.5929

'0,"No error"\n,'

In [10]:
counter.write('*RST;*CLS') 

(24, <StatusCode.success: 0>)

In [95]:
counter.close()